In [1]:
url = r"sqlite:///C:/Users/blinklet/Documents/Chinook_Sqlite.sqlite"

In [2]:
from sqlalchemy import create_engine

engine = create_engine(url)

In [3]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

Base = automap_base()
Base.prepare(autoload_with=engine)

In [4]:
Album = Base.classes.Album
Artist = Base.classes.Artist
Customer = Base.classes.Customer
Employee = Base.classes.Employee
Genre = Base.classes.Genre
Invoice = Base.classes.Invoice
InvoiceLine = Base.classes.InvoiceLine
MediaType = Base.classes.MediaType
Playlist = Base.classes.Playlist
Track = Base.classes.Track
playlisttrack = Base.metadata.tables['PlaylistTrack']

In [21]:
from sqlalchemy import select

statement = (select(Artist))

In [22]:
print(statement)

SELECT "Artist"."ArtistId", "Artist"."Name" 
FROM "Artist"


In [23]:
import pandas as pd

artists = pd.read_sql_query(sql=statement, con=engine)

print(artists.shape)
print(artists.head(5))

(275, 2)
   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains


In [29]:
statement = (select(Artist).where(Artist.Name=='Alice In Chains'))
print(statement)
print()
from sqlalchemy.dialects import sqlite 
query = statement.compile(dialect=sqlite.dialect(),compile_kwargs={"literal_binds": True})
print(query) # will print the compiled query statement againt the dialect.


SELECT "Artist"."ArtistId", "Artist"."Name" 
FROM "Artist" 
WHERE "Artist"."Name" = :Name_1

SELECT "Artist"."ArtistId", "Artist"."Name" 
FROM "Artist" 
WHERE "Artist"."Name" = 'Alice In Chains'


In [48]:
statement = (
    select(Artist)
    .order_by(Artist.Name)
    .limit(5)
    )

alpha_artists = pd.read_sql_query(sql=statement, con=engine)
print(alpha_artists.shape)
print(alpha_artists.head(5))

(5, 2)
   ArtistId                                               Name
0        43                                       A Cor Do Som
1         1                                              AC/DC
2       230          Aaron Copland & London Symphony Orchestra
3       202                                     Aaron Goldberg
4       214  Academy of St. Martin in the Fields & Sir Nevi...


In [51]:
statement = (
    select(Artist)
    .order_by(func.random())
    .limit(5))

rand_artists = pd.read_sql_query(sql=statement, con=engine)
print(rand_artists.shape)
print(rand_artists.head(5))

(5, 2)
   ArtistId                         Name
0        85                Frank Sinatra
1        16               Caetano Veloso
2       265                 Julian Bream
3       203  Nicolaus Esterhazy Sinfonia
4        93                          JET


In [53]:
statement = (select(Album, Track, Artist)
     .join(Track)
     .join(Artist)
    )
print(statement)

SELECT "Album"."AlbumId", "Album"."Title", "Album"."ArtistId", "Track"."TrackId", "Track"."Name", "Track"."AlbumId" AS "AlbumId_1", "Track"."MediaTypeId", "Track"."GenreId", "Track"."Composer", "Track"."Milliseconds", "Track"."Bytes", "Track"."UnitPrice", "Artist"."ArtistId" AS "ArtistId_1", "Artist"."Name" AS "Name_1" 
FROM "Album" JOIN "Track" ON "Album"."AlbumId" = "Track"."AlbumId" JOIN "Artist" ON "Artist"."ArtistId" = "Album"."ArtistId"


In [56]:

statement = (select(Album.Title.label("Album"),
            Artist.Name.label("Artist"),
            Track.Name.label("Track"),
            Track.Composer, 
            Track.Milliseconds.label("Length"))
     .join(Track)
     .join(Artist)
    )
print(statement)

SELECT "Album"."Title" AS "Album", "Artist"."Name" AS "Artist", "Track"."Name" AS "Track", "Track"."Composer", "Track"."Milliseconds" AS "Length" 
FROM "Album" JOIN "Track" ON "Album"."AlbumId" = "Track"."AlbumId" JOIN "Artist" ON "Artist"."ArtistId" = "Album"."ArtistId"


In [73]:
statement = select(
    Employee.EmployeeId, 
    Employee.FirstName.label("Emp_First_Name"),
    Employee.LastName.label("Emp_Last_Name"), 
    Employee.Title,
    Customer.FirstName.label("Cust_First_Name"),
    Customer.LastName.label("Cust_Last_Name"),
    Customer.Company.label("Cust_Company"),   
).outerjoin(Customer)

dataframe = pd.read_sql_query(sql=statement, con=engine)
print(dataframe.shape)
display(dataframe.head().style.hide(axis="index"))

(64, 7)


EmployeeId,Emp_First_Name,Emp_Last_Name,Title,Cust_First_Name,Cust_Last_Name,Cust_Company
1,Andrew,Adams,General Manager,None,None,None
2,Nancy,Edwards,Sales Manager,None,None,None
3,Jane,Peacock,Sales Support Agent,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.
3,Jane,Peacock,Sales Support Agent,François,Tremblay,None
3,Jane,Peacock,Sales Support Agent,Roberto,Almeida,Riotur


In [80]:
statement = (
    select(
        Employee.EmployeeId, 
        (Employee.FirstName + ' ' + Employee.LastName).label("Employee Name"), 
        Employee.Title,
        func.count(Customer.CustomerId).label("Num_Customers")
    )
    .outerjoin(Customer)
    .group_by(Employee.EmployeeId)
)

dataframe = pd.read_sql_query(sql=statement, con=engine)
print(dataframe.to_string(index=False))

 EmployeeId    Employee Name               Title  Num_Customers
          1     Andrew Adams     General Manager              0
          2    Nancy Edwards       Sales Manager              0
          3     Jane Peacock Sales Support Agent             21
          4    Margaret Park Sales Support Agent             20
          5    Steve Johnson Sales Support Agent             18
          6 Michael Mitchell          IT Manager              0
          7      Robert King            IT Staff              0
          8   Laura Callahan            IT Staff              0
